<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="ITM">
</head>

<img src="https://maua.br/images/selo-60-anos-maua.svg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->



# **P1 - RA PAR**

In [ ]:
#@markdown Nome completo e RA (*\<nome\>, \<RA\>*)
Aluno = 'Lucas Miguel de Matos Negri 19.00386-2 ' #@param {type:"string"}






In [1]:
path = 'https://github.com/Rogerio-mack/work/raw/main/data/'

# **Case**: Airbnb

Neste case, analise dados sobre imóveis alugados pela plataforma Airbnb em NYC no ano de 2019.


In [4]:
import pandas as pd


df = pd.read_csv(path + 'AB_NYC_2019.csv')
df.head()

id                                            name  host_id  \
0   20913              Charming 1 bed GR8 WBurg LOCATION!    79402   
1   25235               Large 2 Bedroom Great for Groups!    87773   
2   76761  Eveland the Place to Stay & Enjoy a 5-⭐️ 2bdrm   118971   
3  135465             Garden apartment close to Manhattan   663879   
4  169002                Modern Space in Charming Pre-war   805344   

     host_name neighbourhood_group       neighbourhood  latitude  longitude  \
0   Christiana            Brooklyn        Williamsburg  40.70984  -73.95775   
1       Shelly            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   
2       Evelyn            Brooklyn         South Slope  40.66552  -73.99019   
3  Christopher            Brooklyn         Fort Greene  40.68626  -73.97598   
4         Alec           Manhattan              Harlem  40.82411  -73.94934   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0  Entire home/apt    100               5                168  2018-07-22   
1  Entire home/apt    125              90                162  2019-06-28   
2  Entire home/apt    169               2                398  2019-06-28   
3  Entire home/apt    170               3                 86  2019-06-20   
4     Private room     65               2                 41  2019-06-16   

   reviews_per_month  calculated_host_listings_count  availability_365  
0               1.57                               1                 0  
1               1.46                               2               137  
2               3.97                               3               182  
3               0.91                               2               286  
4               0.43                               2                59

# Q1. Proprietários com mais imóveis na plataforma

Obtenha os 3 proprietários ("hosts") com mais imóveis sendo anunciados na plataforma, e a respectiva quantidade de imóveis anunciados.


In [18]:

top3_hosts = df[df.calculated_host_listings_count>0].groupby('host_name').calculated_host_listings_count.sum().sort_values(ascending=False).head(3)

print(top3_hosts)

host_name
Sonder (NYC)    2289
Kara             484
Blueground       464
Name: calculated_host_listings_count, dtype: int64


# Q2A. Locatários profissionais $\times$ anunciantes com um único imóvel

Considere o preço médio dos imóveis. Qual a diferença do preço médio dos imóveis dos anunciantes com mais de um imóvel e dos anunciantes com um único imóvel anunciado na plataforma?

In [39]:
host_more = df['host_id'].value_counts()[df['host_id'].value_counts()>1]
host_less = df['host_id'].value_counts()[df['host_id'].value_counts()==1]
price = df['price']
price_more = price[df['host_id'].isin(host_more.index)]
price_less = price[df['host_id'].isin(host_less.index)]
preco_mais_de_um_imovel = price_more.mean()
preco_mais_um_imovel = price_less.mean()
print( f'Diferença de preços: USD {preco_mais_de_um_imovel - preco_mais_um_imovel:.2f}')

Diferença de preços: USD 67.81


# Q3A.  Manhattan *outliers*

Qual o percentual de imóveis de Manhattan com preços estatisticamente discrepantes (*outliers*) com relação aos preços de todos os imóveis?



In [51]:
q1 = df[df.neighbourhood_group=='Manhattan'].price.quantile(0.25)
q3 = df[df.neighbourhood_group=='Manhattan'].price.quantile(0.75)
iqr = q3 - q1
sup = df[df.neighbourhood_group=='Manhattan'].price > q3 +(1.5*iqr)
inf = df[df.neighbourhood_group=='Manhattan'].price < q1 -(1.5*iqr)
outliers = df[df.neighbourhood_group=='Manhattan'].price[sup | inf]
man_price = df[df.neighbourhood_group=='Manhattan'].price
perc_Manhattan = (len(outliers)/len(man_price))*100


print( f'Percentuais de outliers de preços, Manhattan: {perc_Manhattan:.2f} %')


Percentuais de outliers de preços, Manhattan: 7.34 %


# Q4A. Preços $\times$ Descrições

Considere as descrições dos anúncios em `df.name`. Crie uma coluna `df['word']` que contém 'beautiful','sunny','spacious' ou 'heart' de acordo com a presença dessa palavra no anúncio. Caso não tenha nenhuma das palavras, assinale 'others'. Ao final, verifique a quantidade e o preço médio dos imóveis para cada grupo dessas palavras nos anúncios.

Atenção, considere a ordem 'beautiful','sunny','spacious' ou 'heart' (isto é, se o texto contém duas ou mais palavras, por exemplo 'beautiful' e 'sunny', considere 'beautiful'). Considere também quaisquer combinações de maiúsculas/minúsculas nas palavras.

In [121]:

def cat_name(name):
    name = str(name).lower()
    if 'beautiful' in name:
        return 'beautiful'
    elif 'sunny' in name:
        return 'sunny'
    elif 'spacious' in name:
        return 'spacious'
    elif 'heart' in name:
        return 'heart'
    else:
        return 'others'

df['word'] = df['name'].apply(cat_name)

word_stats = df.groupby('word').agg({'price': [ 'mean']})

word_stats

price
                 mean
word                 
beautiful  137.926829
heart      166.388889
others     157.886218
spacious   128.516667
sunny      137.674419

# Q5. Enriquecendo os dados

Enriqueça os dados dos imóveis com os valores dos impostos e multas municipais. Cacule a média de impostos e multas de cada grupo de vizinhaça, `neighbourhood_group`. Note que existem imóveis sem impostos e sem multas, esses são imóveis isentos, mas devem ser considerados nos cálculos de média (por exemplo, você pode considerar esses imóveis com valor 0).

In [122]:
df_taxes = pd.read_csv(path + 'AB_NYC_2019_taxes.csv')
df_taxes

id   type   value
0     35907127  taxes  1315.0
1       804815  taxes     NaN
2     22328585  taxes   900.0
3     27876600  taxes   869.0
4     34976037  taxes  1629.0
...        ...    ...     ...
1523  34284316  fines     NaN
1524  20325804  fines     NaN
1525  25872986  fines     NaN
1526  19365218  fines   906.0
1527   4235843  fines     NaN

[1528 rows x 3 columns]

In [153]:

df = pd.read_csv(path + 'AB_NYC_2019.csv')
df_taxes = pd.read_csv(path + 'AB_NYC_2019_taxes.csv')
df = pd.merge(df, df_taxes, on='id')

df.head()


id                                            name  host_id   host_name  \
0  20913              Charming 1 bed GR8 WBurg LOCATION!    79402  Christiana   
1  20913              Charming 1 bed GR8 WBurg LOCATION!    79402  Christiana   
2  25235               Large 2 Bedroom Great for Groups!    87773      Shelly   
3  25235               Large 2 Bedroom Great for Groups!    87773      Shelly   
4  76761  Eveland the Place to Stay & Enjoy a 5-⭐️ 2bdrm   118971      Evelyn   

  neighbourhood_group       neighbourhood  latitude  longitude  \
0            Brooklyn        Williamsburg  40.70984  -73.95775   
1            Brooklyn        Williamsburg  40.70984  -73.95775   
2            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   
3            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   
4            Brooklyn         South Slope  40.66552  -73.99019   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0  Entire home/apt    100               5                168  2018-07-22   
1  Entire home/apt    100               5                168  2018-07-22   
2  Entire home/apt    125              90                162  2019-06-28   
3  Entire home/apt    125              90                162  2019-06-28   
4  Entire home/apt    169               2                398  2019-06-28   

   reviews_per_month  calculated_host_listings_count  availability_365   type  \
0               1.57                               1                 0  taxes   
1               1.57                               1                 0  fines   
2               1.46                               2               137  taxes   
3               1.46                               2               137  fines   
4               3.97                               3               182  taxes   

    value  
0   789.0  
1   796.0  
2  1779.0  
3     NaN  
4  1516.0

# Q6. Quebras

Construa no `Pandas` um "relatório" (`df_report`) como abaixo. Os valores são ordenados `neighbourhood_group`,`neighbourhood`. Cada "página" contém apenas 30 linhas. A cada "quebra" de `neighbourhood_group` é gerada uma linha com preço médio do `neighbourhood_group` e, então, é iniciada uma nova página.

O item `line_id` é opcional mas deve auxiliá-lo nessa tarefa.

Ao final produza a soma dos preços das páginas abaixo...

In [ ]:
sum_price_pages = df_report[ df_report.page_id.isin([1,24,25,28]) ].groupby('page_id').price.sum().astype(int).sum()

print(f'A soma dos preços das páginas 1, 24, 25 e 28 é: {sum_price_pages}')

#### Aqui um *sample* do report.

In [ ]:
df_report.iloc[-10::,:]

neighbourhood_group  neighbourhood       price  page_id  line_id
795              Queens       Woodside   75.000000       28       17
796              Queens       Woodside  120.000000       28       18
797              Queens           MEAN   92.227848       28       19
798       Staten Island  Randall Manor   99.000000       29        1
799       Staten Island    South Beach   20.000000       29        2
800       Staten Island    South Beach  100.000000       29        3
801       Staten Island     St. George  130.000000       29        4
802       Staten Island      Todt Hill  135.000000       29        5
803       Staten Island  Tompkinsville   59.000000       29        6
804       Staten Island           MEAN   89.428571       29        7